### Data preprocessing

In [1]:
import pandas as pd
import re
from pathlib import Path

In [2]:
# Define the project root
ROOT = Path.cwd().parent

In [3]:
trade_data = pd.read_csv(ROOT / "data" / "raw" / "TradesList.txt", sep='\t')
trade_data.head()

,Symbol,Trade Type,Entry DateTime,Exit DateTime,Entry Price,Exit Price,Trade Quantity,Max Open Quantity,Max Closed Quantity,Profit/Loss (C),...,Exit Efficiency,Total Efficiency,Commission (C),High Price While Open,Low Price While Open,Note,Open Position Quantity,Close Position Quantity,Duration,Account
0,[Sim]MCLN25-NYMEX (Sim2),Short,2025-05-21 14:33:36.209 BP,2025-05-21 15:20:18.249 EP,62.09,61.92,14.0,14.0,14.0,216.60,...,56.7%,39.6%,22.4,62.14,61.71,SimpleBracket.twconfig,14.0,0.0,00:46:42,Sim2
1,[Sim]MCLN25-NYMEX,Long,2025-05-22 14:01:30.798 BP,2025-05-22 14:05:19.642 EP,60.54,60.46,14.0,14.0,14.0,-134.40,...,-6.7%,-106.7%,22.4,60.54,60.45,SimpleBracket.twconfig,14.0,0.0,00:03:48,Sim2
2,[Sim]MCLN25-NYMEX,Long,2025-05-22 14:06:19.134 BP,2025-05-22 14:17:09.188 EP,60.48,60.68,14.0,14.0,14.0,263.60,...,34.9%,10.9%,22.4,60.80,60.46,SimpleBracket.twconfig,14.0,0.0,00:10:50,Sim2
3,[Sim]MCLN25-NYMEX,Short,2025-05-22 14:17:09.188 BP,2025-05-22 14:17:11.028 EP,60.79,60.80,2.0,2.0,2.0,-5.20,...,-80.0%,-130.0%,3.2,60.80,60.78,SimpleBracket.twconfig,2.0,0.0,00:00:01,Sim2
4,[Sim]MCLN25-NYMEX,Short,2025-05-22 14:22:07.151 BP,2025-05-22 14:23:08.378 EP,60.93,61.01,14.0,14.0,14.0,-134.40,...,4.0%,-96.0%,22.4,61.03,60.93,SimpleBracket.twconfig,14.0,0.0,00:01:01,Sim2


In [4]:
trade_data.columns

Index(['Symbol', 'Trade Type', 'Entry DateTime', 'Exit DateTime',
       'Entry Price', 'Exit Price', 'Trade Quantity', 'Max Open Quantity',
       'Max Closed Quantity', 'Profit/Loss (C)', 'Cumulative Profit/Loss (C)',
       'FlatToFlat Profit/Loss (C)', 'FlatToFlat Max Open Profit (C)',
       'FlatToFlat Max Open Loss (C)', 'Max Open Profit (C)',
       'Max Open Loss (C)', 'Entry Efficiency', 'Exit Efficiency',
       'Total Efficiency', 'Commission (C)', 'High Price While Open',
       'Low Price While Open', 'Note', 'Open Position Quantity',
       'Close Position Quantity', 'Duration', 'Account'],
      dtype='object')

In [5]:
trade_data.drop(columns=['Max Open Quantity',
                         'Max Closed Quantity',
                         'Cumulative Profit/Loss (C)',
                         'FlatToFlat Profit/Loss (C)',
                         'FlatToFlat Max Open Profit (C)',
                         'FlatToFlat Max Open Loss (C)',
                         'Max Open Profit (C)',
                         'Max Open Loss (C)',
                         'Entry Efficiency',
                         'Exit Efficiency',
                         'Total Efficiency',
                         'Note',
                         'Open Position Quantity',
                         'Close Position Quantity'],
                         inplace=True)

In [6]:
# Drop empty rows
trade_data.dropna(axis=0, inplace=True)

In [7]:
trade_data.columns

Index(['Symbol', 'Trade Type', 'Entry DateTime', 'Exit DateTime',
       'Entry Price', 'Exit Price', 'Trade Quantity', 'Profit/Loss (C)',
       'Commission (C)', 'High Price While Open', 'Low Price While Open',
       'Duration', 'Account'],
      dtype='object')

In [8]:
# Rename columns
trade_data.rename(columns={'Symbol': 'symbol',
                            'Trade Type': 'trade_type',
                            'Entry DateTime': 'entry_datetime',
                            'Exit DateTime': 'exit_datetime',
                            'Entry Price': 'entry_price',
                            'Exit Price': 'exit_price',
                            'Trade Quantity': 'quantity',
                            'Profit/Loss (C)': 'profit_loss',
                            'Commission (C)': 'commission',
                            'High Price While Open': 'price_range_high',
                            'Low Price While Open': 'price_range_low',
                            'Duration': 'duration'}, inplace=True)


In [9]:
# Clean the symbol string
trade_data['symbol'] = trade_data['symbol'].apply(
    lambda x: x.split(']')[-1].split('-')[0].strip().split(' ')[0][0:3]
)

In [10]:
# Convert datetime columns to datetime type
trade_data['entry_datetime'] = pd.to_datetime(
    trade_data['entry_datetime'].str.replace(' BP', '', regex=False)
)

trade_data['exit_datetime'] = pd.to_datetime(
    trade_data['exit_datetime'].str.replace(' EP', '', regex=False)
)

In [11]:
# Convert duration column to timedelta type
trade_data['duration_sec'] = pd.to_timedelta(trade_data['duration']).dt.total_seconds()
trade_data.drop(columns=['duration'], inplace=True)

In [12]:
# Examine final DataFrame
trade_data.head()

,symbol,trade_type,entry_datetime,exit_datetime,entry_price,exit_price,quantity,profit_loss,commission,price_range_high,price_range_low,Account,duration_sec
0,MCL,Short,2025-05-21 14:33:36.209,2025-05-21 15:20:18.249,62.09,61.92,14.0,216.60,22.4,62.14,61.71,Sim2,2802.0
1,MCL,Long,2025-05-22 14:01:30.798,2025-05-22 14:05:19.642,60.54,60.46,14.0,-134.40,22.4,60.54,60.45,Sim2,228.0
2,MCL,Long,2025-05-22 14:06:19.134,2025-05-22 14:17:09.188,60.48,60.68,14.0,263.60,22.4,60.80,60.46,Sim2,650.0
3,MCL,Short,2025-05-22 14:17:09.188,2025-05-22 14:17:11.028,60.79,60.80,2.0,-5.20,3.2,60.80,60.78,Sim2,1.0
4,MCL,Short,2025-05-22 14:22:07.151,2025-05-22 14:23:08.378,60.93,61.01,14.0,-134.40,22.4,61.03,60.93,Sim2,61.0


In [13]:
# Save the cleaned DataFrame to a new CSV file
trade_data.to_csv(ROOT / "data" / "processed" / "trades.csv", index=False)